In [54]:
import pandas as pd
import time
import pickle
import datetime
from sqlalchemy import create_engine

from sqlalchemy_utils import database_exists, create_database
import psycopg2
import pandas as pd
import csv
import matplotlib.pyplot as plt

from collections import OrderedDict
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import numpy as np
from sklearn import metrics

from geopy.distance import great_circle
from shapely.geometry import MultiPoint

In [5]:
pkfile = open('regressor_Sep26.pk1','rb')
regressor = pickle.load(pkfile)
pkfile.close()

In [8]:
regressor.coef_

array([-0.00067977, -0.00049957, -0.00183946, -0.01591451,  0.10833862,
       -0.15959181])

In [10]:
pk1_file = open('cluster_monthly_observation_data_2018-07-31_2019-08-31.pk1','rb')
observation_data = pickle.load(pk1_file)
pk2_file = open('formatted_climate_data_2018-03_2019-08.pk1','rb')
climate_data = pickle.load(pk2_file)

In [11]:
def get_dragon_butter_ratio_i_j(data,i,j):
    butterfly_num = data[i]['data']['butterfly'].loc[j]['monthly_count']
    dragonfly_num = data[i]['data']['dragonfly'].loc[j]['monthly_count']    
    val = butterfly_num + 1
    val = dragonfly_num / val
    return val

def get_month_ppt_i_j(data,i,j):
    val = data[i]['ppts'][j]
    return val

def get_month_tmean_i_j(data,i,j):
    val = data[i]['tmeans'][j]
    return val

def get_features_clusterI_monthJ(data_climate,data_wildlife,i,j):
    ppt_m1 = get_month_ppt_i_j(data_climate,i,j-3)
    ppt_m2 = get_month_ppt_i_j(data_climate,i,j-2)
    ppt_m3 = get_month_ppt_i_j(data_climate,i,j-1)
    tmean_m1 = get_month_tmean_i_j(data_climate,i,j-3)
    tmean_m2 = get_month_tmean_i_j(data_climate,i,j-2)
    tmean_m3 = get_month_tmean_i_j(data_climate,i,j-1)    
    depend_var = get_dragon_butter_ratio_i_j(data_wildlife,i,j)
    returndict = {'depend_var':depend_var,'ppt_m1':ppt_m1,'ppt_m2':ppt_m2,'ppt_m3':ppt_m3,'tmean_m1':tmean_m1,'tmean_m2':tmean_m2,'tmean_m3':tmean_m3}
    return returndict

def get_features_monthJ(data_climate,data_wildlife,j):
    val = []
    for i in range(len(data_wildlife)-1):
    #for i in range(30):    
        valdict = get_features_clusterI_monthJ(data_climate,data_wildlife,i,j)
        val.append(valdict)
    return val

In [23]:
def get_features(data_climate,data_wildlife):
    val = []
    for i in range(len(data_wildlife)-1):
        for j in range(13):
            valdict = get_features_clusterI_monthJ(data_climate,data_wildlife,i,j)
            val.append(valdict)
    return val
            

In [24]:
fullset_df = pd.DataFrame(get_features(climate_data,observation_data))

In [25]:
fullset_df

,depend_var,ppt_m1,ppt_m2,ppt_m3,tmean_m1,tmean_m2,tmean_m3
0,0.057692,175.725006,181.679001,126.203003,20.050001,22.970001,25.943001
1,0.226415,181.679001,126.203003,116.495003,22.970001,25.943001,7.101000
2,0.036145,126.203003,116.495003,170.445007,25.943001,7.101000,10.266001
3,0.250000,116.495003,170.445007,126.964996,7.101000,10.266001,22.799002
4,0.125000,170.445007,126.964996,188.686996,10.266001,22.799002,25.460001
...,...,...,...,...,...,...,...
476,0.037879,196.076004,150.774002,26.639000,26.318001,26.549002,19.731001
477,0.156171,150.774002,26.639000,182.569000,26.549002,19.731001,9.831000
478,0.067039,26.639000,182.569000,242.429993,19.731001,9.831000,8.925000
479,0.200000,182.569000,242.429993,186.496002,9.831000,8.925000,7.580000


In [12]:
makedataframe12 = pd.DataFrame(get_features_monthJ(climate_data,observation_data,12))

In [14]:
makedataframe12.describe()

,depend_var,ppt_m1,ppt_m2,ppt_m3,tmean_m1,tmean_m2,tmean_m3
count,37.000000,37.000000,37.000000,37.000000,37.000000,37.000000,37.000000
mean,0.241810,145.167649,118.712296,132.809514,4.262865,1.288514,4.253649
std,0.271273,69.454972,31.207719,65.805401,2.690482,4.007221,4.737629
min,0.000958,33.110001,81.075996,32.317001,0.209000,-5.846000,-3.557000
25%,0.078481,86.230003,96.310997,90.501999,1.966000,-2.098000,0.971000
50%,0.137251,143.160995,111.913002,103.193001,3.665000,0.806000,2.834000
75%,0.233227,172.371994,132.654999,175.710999,6.490000,4.885000,7.698000
max,0.933673,292.768005,196.330994,311.632996,10.571000,9.530001,14.546000


In [16]:
X = makedataframe12[['ppt_m1','ppt_m2','ppt_m3','tmean_m1','tmean_m2','tmean_m3']]
y = makedataframe12['depend_var']

In [17]:
score = regressor.score(X,y)
score

-2.407181251337997

In [19]:
makedataframe1 = pd.DataFrame(get_features_monthJ(climate_data,observation_data,1))
X1 = makedataframe1[['ppt_m1','ppt_m2','ppt_m3','tmean_m1','tmean_m2','tmean_m3']]
y1 = makedataframe1['depend_var']

In [20]:
score = regressor.score(X1,y1)
score

-115.81457728401273

In [21]:
df = pd.DataFrame({'Actual': y1, 'Predicted': regressor.predict(X1)})

In [26]:
df.head()

,Actual,Predicted
0,0.226415,1.969976
1,0.177711,2.795256
2,0.020000,2.359957
3,0.010638,2.340036
4,0.039443,2.921377


In [27]:
X_full = fullset_df[['ppt_m1','ppt_m2','ppt_m3','tmean_m1','tmean_m2','tmean_m3']]
y_full = fullset_df['depend_var']

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X_full, y_full, test_size=0.2, random_state=0)

In [29]:
regressor_full = LinearRegression()
regressor_full.fit(X_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [31]:
coeff_df = pd.DataFrame(regressor_full.coef_, X_full.columns, columns=['Coefficient'])
coeff_df

,Coefficient
ppt_m1,-0.000230
ppt_m2,0.000222
ppt_m3,-0.000266
tmean_m1,-0.004706
tmean_m2,-0.002057
tmean_m3,-0.008314


In [32]:
fullset_df.describe()

,depend_var,ppt_m1,ppt_m2,ppt_m3,tmean_m1,tmean_m2,tmean_m3
count,481.000000,481.000000,481.000000,481.000000,481.000000,481.000000,481.000000
mean,0.158543,136.721784,136.749784,134.242160,17.237506,15.831764,14.443412
std,0.279411,61.178345,60.515263,59.878031,8.092116,9.129551,9.482151
min,0.000000,24.924999,24.924999,24.924999,0.209000,-5.846000,-5.846000
25%,0.000000,92.063004,93.255997,92.063004,10.146001,7.101000,5.523000
50%,0.058824,128.671997,128.065994,126.084999,20.571001,20.130001,14.904000
75%,0.186386,167.664993,167.664993,166.574005,23.930000,23.928001,23.490002
max,2.418605,392.566986,392.566986,392.566986,27.733002,27.733002,27.733002


In [33]:
y_pred = regressor_full.predict(X_test)

In [40]:
score = regressor_full.score(X_full,y_full)
score

0.18909240341533884

In [41]:
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test,y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Mean Absolute Error: 0.1276119874858841
Mean Squared Error: 0.03820035643574603
Root Mean Squared Error: 0.19544911469675688


In [42]:
score = regressor_full.score(X_test,y_test)
score

0.3631406038524868

In [43]:
score = regressor.score(X_test,y_test)
score

-18.19274179123474

In [44]:
output = open('regressor_allmonths_Sep26.pk1','wb')
pickle.dump(regressor_full,output)
output.close()

In [45]:
cluster_coords = pd.read_csv('cluster_coords.csv', index_col=0)
cluster_coords = cluster_coords.reset_index()

In [94]:
def get_closest_cluster_point(clusters,reference):
    #print(clusters)
    closest_point = min(clusters, key=lambda point: great_circle(point,reference).m)
    return tuple(closest_point)

In [96]:
cluster_array = [(37.358594, -88.919156), (41.80800418, -87.58900703), (39.4532783639, -87.4155096884), (39.4410783333, -88.1627361667), (42.2927100677, -85.148530556), (39.4712017408, -88.1936910734), (40.9798075632, -81.3047336788), (41.1012884709, -83.1792231645), (38.7248236493, -83.1785495849), (39.5817333333, -82.0146111167), (40.699795, -82.53702), (40.3729181, -82.3967142), (39.1298138333, -84.5002366667), (38.9642299915, -77.3278066023), (41.326536748, -81.4385604858), (38.7709536727, -77.2703889733), (38.94863, -77.3300333333), (37.7559179311, -83.9149260521), (36.6652793884, -84.4941940308), (39.5883851759, -79.9864799835), (38.7814325094, -76.710026823), (39.0982421491, -76.7286836462), (39.09078154, -77.12346602), (39.1265712307, -76.8781313699), (39.0171668581, -76.7995197217), (35.7945046404, -78.68952847), (35.5280715306, -82.3568541754), (35.8827725, -78.9871598), (35.8057043825, -79.0707965844), (35.8133333333, -81.1672222222), (34.8841629028, -82.2802352905), (33.2621033333, -87.5149916667), (33.4223913032, -86.7678254792), (34.419152039, -85.616389241), (34.2233266246, -86.161356589), (32.0110664637, -85.4421526885), (32.788461, -86.871655), (33.4582361919, -85.5765152047)]



#cluster_array =[[ 2.28181753, -85.12953758],
#       [ 49.28104873, 85.13474258],
#       [ 13.28251604, 7.13537407]]
reference_point = [41.27763488, -87.13232721]

In [101]:
newtest = get_closest_cluster_point(cluster_array,reference_point)

In [102]:
newtest

(41.80800418, -87.58900703)

In [103]:
newtest[0]

41.80800418

In [108]:
cluster_coords.loc[cluster_coords['lat']==newtest[0]]['id'].iloc[0]

2

In [104]:
cluster_coords.head()

,lat,lon,id
0,37.358594,-88.919156,1
1,41.808004,-87.589007,2
2,39.453278,-87.415510,3
3,39.441078,-88.162736,4
4,42.292710,-85.148531,5


In [109]:
def get_closest_cluster_id(clusters,reference):
    cluster_array = [(37.358594, -88.919156), (41.80800418, -87.58900703), (39.4532783639, -87.4155096884), (39.4410783333, -88.1627361667), (42.2927100677, -85.148530556), (39.4712017408, -88.1936910734), (40.9798075632, -81.3047336788), (41.1012884709, -83.1792231645), (38.7248236493, -83.1785495849), (39.5817333333, -82.0146111167), (40.699795, -82.53702), (40.3729181, -82.3967142), (39.1298138333, -84.5002366667), (38.9642299915, -77.3278066023), (41.326536748, -81.4385604858), (38.7709536727, -77.2703889733), (38.94863, -77.3300333333), (37.7559179311, -83.9149260521), (36.6652793884, -84.4941940308), (39.5883851759, -79.9864799835), (38.7814325094, -76.710026823), (39.0982421491, -76.7286836462), (39.09078154, -77.12346602), (39.1265712307, -76.8781313699), (39.0171668581, -76.7995197217), (35.7945046404, -78.68952847), (35.5280715306, -82.3568541754), (35.8827725, -78.9871598), (35.8057043825, -79.0707965844), (35.8133333333, -81.1672222222), (34.8841629028, -82.2802352905), (33.2621033333, -87.5149916667), (33.4223913032, -86.7678254792), (34.419152039, -85.616389241), (34.2233266246, -86.161356589), (32.0110664637, -85.4421526885), (32.788461, -86.871655), (33.4582361919, -85.5765152047)]


    closest_point = min(cluster_array, key=lambda point: great_circle(point,reference).m)

    cluster_coords = pd.read_csv('cluster_coords.csv', index_col=0)
    cluster_coords = cluster_coords.reset_index()
    cluster_id = cluster_coords.loc[cluster_coords['lat']==closest_point[0]]['id'].iloc[0]

    return cluster_id

In [ ]:
def get_prediction_butterfly(id):
    pk2_climatefile = open('formatted_climate_data_2018-03_2019-08.pk1','rb')
    climate_data = pickle.load(pk2_climatefile)